In [0]:
package com.databricks.dais2025

import java.sql.Timestamp

object MyStructs {
  case class Input(
      sensor_id: String, // ID of the sensor
      location: String, // Location name
      city: String, // City name
      timestamp: Timestamp, // Timestamp of the reading
      temperature: Double, // Current temperature
      humidity: Double, // Current humidity
      co2_level: Double, // Current CO2 level
      pm25_level: Double // Current PM2.5 level
  )

  case class State(
      temperature: Double,
      timestamp: Timestamp,
      humidity: Double,
      co2_level: Double,
      pm25_level: Double,
      location: String)

  case class Output(
      sensor_id: String, // ID of the sensor
      location: String, // Location name
      city: String, // City name
      timestamp: Timestamp, // Timestamp of the reading
      temperature: Double, // Current temperature
      humidity: Double, // Current humidity
      co2_level: Double, // Current CO2 level
      pm25_level: Double, // Current PM2.5 level
      hourly_avg_temp: Double, // Average temperature in the last hour
      daily_avg_temp: Double, // Average temperature in the last day
      temperature_trend: String, // Trend of temperature (rising/falling/stable)
      high_temp_count: Int, // Count of high temperature readings
      alerts: String // Alerts as a semicolon-separated string
  )
}

In [0]:
package com.databricks.dais2025.tws 

import com.databricks.dais2025.MyStructs._
import org.apache.spark.sql.streaming.StatefulProcessor
import org.apache.spark.sql.streaming.ListState
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.streaming.TimeMode
import org.apache.spark.sql.streaming.TTLConfig
import org.apache.spark.sql.streaming.TimerValues
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.internal.SQLConf

class EnvironmentalMonitorListProcessor extends StatefulProcessor[String, Input, Output] {

  private val thresholds = Map(
    "temperature" -> 25.0,
    "humidity" -> 80.0,
    "co2_level" -> 1000.0,
    "pm25_level" -> 35.0
  )

  private var readingsState: ListState[State] = _

  override def init(outputMode: OutputMode, timeMode: TimeMode): Unit = {
    readingsState = getHandle.getListState(
      "city_readings", Encoders.product[State], TTLConfig.NONE)
  }

  override def handleInputRows(
      city: String,
      rows: Iterator[Input],
      timerValues: TimerValues
  ): Iterator[Output] = {

    val processedResults = rows.flatMap { row =>
      try {
        val currentReading = new State(
          row.temperature,
          row.timestamp,
          row.humidity,
          row.co2_level,
          row.pm25_level,
          row.location
        )
        if (row.temperature > thresholds("temperature")) {
          readingsState.appendValue(
            currentReading
          )
        }

        val validReadings = readingsState.get().toList

        val allReadings = validReadings :+ currentReading
        val avgTemp =
          if (allReadings.nonEmpty) allReadings.map(_.temperature).sum / allReadings.size
          else row.temperature

        val trend = validReadings.sortBy(_.timestamp.getTime).lastOption match {
          case Some(lastReading) =>
            val lastTemp = lastReading.temperature
            if (row.temperature > lastTemp) "rising"
            else if (row.temperature < lastTemp) "falling"
            else "stable"
          case None => "stable"
        }

        val sb = new StringBuilder()
        if (row.temperature > thresholds("temperature")) {
          sb ++= s"High temperature alert: ${row.temperature} °C;"
        }
        if (row.humidity > thresholds("humidity")) {
          sb ++= s"High humidity alert: ${row.humidity} %;"
        }
        if (row.co2_level > thresholds("co2_level")) {
          sb ++= s"High co2_level alert: ${row.co2_level} ppm;"
        }
        if (row.co2_level > thresholds("pm25_level")) {
          sb ++= s"High pm25_level alert: ${row.co2_level} µg/m³;"
        }

        val alerts = sb.toString()

        List(Output(
          row.sensor_id,
          row.location,
          row.city,
          row.timestamp,
          row.temperature,
          row.humidity,
          row.co2_level,
          row.pm25_level,
          avgTemp,
          avgTemp,
          trend,
          validReadings.size,
          alerts))
      } catch {
        case e: Exception =>
          // scalastyle:off println
          println(s"Error processing row: ${e.getMessage}")
          // scalastyle:on println
          List.empty
      }
    }.toList

    processedResults.iterator
  }
}
